In [39]:
import time
import json
import requests

from selenium import webdriver
from bs4 import BeautifulSoup

from tqdm import tqdm
import pandas as pd
from datetime import datetime

# Get table details

## Table headers

In [ ]:

url = 'https://services1.arcgis.com/zBDE2yVH0hTpllh3/arcgis/rest/services/seguimiento_COVID_V3/FeatureServer/0?f=json'

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract and print the list of fields
    fields = data.get('fields', [])
    for field in fields:
        #print(f"Name: {field['name']}, Type: {field['type']}, Alias: {field['alias']}")
        print(field['name'])
else:
    print("Failed to retrieve metadata from the service.")


## Tables unique values

In [26]:
base_url = 'https://services1.arcgis.com/zBDE2yVH0hTpllh3/arcgis/rest/services/seguimiento_COVID_V3/FeatureServer/0/query'

# Function to get unique values for a field
def get_unique_values(field_name):
    params = {
        'f': 'json',
        'where': '1=1',  # A condition that's always true to get all features
        'outFields': field_name,
        'returnDistinctValues': 'true',
        'returnGeometry': 'false'
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract unique values
    unique_values = set()
    for feature in data.get('features', []):
        attributes = feature.get('attributes', {})
        value = attributes.get(field_name)
        if value is not None:
            unique_values.add(value)
    
    return list(unique_values)

capacidad_values = get_unique_values('Capacidad')
departamento_values = get_unique_values('Departamento')
nombre_municipio_values = get_unique_values('Nombre_municipio')

print(f"Unique 'Capacidad' values: {capacidad_values}")
print(f"Unique 'Departamento' values: {departamento_values}")
print(f"Unique 'Nombre_municipio' values: {nombre_municipio_values}")

Unique 'Capacidad' values: ['Adultos', 'Cuidado Intensivo Adulto', 'Cuidado Intermedio Adulto']
Unique 'Departamento' values: ['Total 88 - Archipiélago de San Andrés, Providencia y Santa Catalina', 'Total 91 - Amazonas', '18 - Caquetá', 'Total 85 - Casanare', 'Total 11 - Bogotá, D.C.', '08 - Atlántico', '11 - Bogotá, D.C.', 'Total 50 - Meta', 'Total 86 - Putumayo', 'Total 95 - Guaviare', 'Total 20 - Cesar', 'Total 15 - Boyacá', 'Total 17 - Caldas', 'Total 73 - Tolima', 'Total 19 - Cauca', 'Total 47 - Magdalena', '94 - Guainía', '86 - Putumayo', 'Total 05 - Antioquia', '44 - La Guajira', 'Total 41 - Huila', 'Total -1 - NO DEFINIDO', '73 - Tolima', 'Total 94 - Guainía', 'Total 08 - Atlántico', '68 - Santander', '20 - Cesar', '05 - Antioquia', '25 - Cundinamarca', '41 - Huila', '13 - Bolívar', 'Total 52 - Nariño', 'Total 63 - Quindio', '81 - Arauca', 'Total 25 - Cundinamarca', 'Total 99 - Vichada', '66 - Risaralda', '54 - Norte de Santander', '70 - Sucre', 'Total 18 - Caquetá', '-1 - NO D

# Get info

In [42]:
def format_date(date):
    return date.strftime('%Y-%m-%d %H:%M:%S')

# Function to query the API and return a DataFrame
def query_to_dataframe(start_date, end_date, fields):
    base_url = 'https://services1.arcgis.com/zBDE2yVH0hTpllh3/arcgis/rest/services/seguimiento_COVID_V3/FeatureServer/0/query'
    start_date_formatted = format_date(start_date)
    end_date_formatted = format_date(end_date)

    print(start_date_formatted, end_date_formatted)

    params = {
        'f': 'json',
        'where': f"(Fecha >= timestamp '{start_date_formatted}') AND (Fecha <= timestamp '{end_date_formatted}')",
        'outFields': ','.join(fields),
        'returnGeometry': 'false',
        'outSR': '4326',
        'resultOffset': 0,
        'resultRecordCount': 2000  # We are limited to 2000 lenght tables
    }

    data_flattened = []
    #while True:
    for i in tqdm(range(100000000000000)):
        response = requests.get(base_url, params=params)
        response_json = response.json()
        features = response_json.get('features', [])
        if not features:
            break  # No more data to retrieve
        data_flattened.extend(features)
        params['resultOffset'] += params['resultRecordCount']

    df = pd.DataFrame([feature['attributes'] for feature in data_flattened])

    if 'Fecha' in df.columns:
        df['Fecha'] = pd.to_datetime(df['Fecha'], unit='ms')

    return df

start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)
fields = [
    'Fecha',
    'Concepto_Desc',
    'Capacidad',
    'Departamento',
    'Municipio',
    'COVID_confirmados',
    'COVID_sospechosos',
    'No_COVID',
    'Total_camas',
    'Camas_disponibles',
    'PCD',
    'Total_casos_COVID',
    'Porcentaje_camas_disponibles',
    'Ocupación_total_COVID',
    'Nombre_departamento',
    'Nombre_municipio',
    'PrefijoDpto',
    'PrefijoMpio',
    'FID',
    'ID_CARGA'
]

df = query_to_dataframe(start_date, end_date, fields)

2020-01-01 00:00:00 2023-12-31 00:00:00


  0%|          | 720/100000000000000 [11:37<26901754966:39:46,  1.03it/s]


In [53]:
## Save info
df.to_csv('../output_files/camas_uci_colombia.csv', index=False)

In [52]:
departamentos_colombia = ['08 - Atlántico', '11 - Bogotá, D.C.', '13 - Bolívar',
       '15 - Boyacá', '20 - Cesar', '05 - Antioquia', '25 - Cundinamarca',
       '19 - Cauca', '23 - Córdoba', '27 - Chocó', '17 - Caldas',
       '18 - Caquetá', '86 - Putumayo', '47 - Magdalena', '52 - Nariño',
       '50 - Meta', '73 - Tolima', '85 - Casanare', '68 - Santander',
       '41 - Huila', '94 - Guainía', '63 - Quindio', '70 - Sucre',
       '76 - Valle del Cauca', '66 - Risaralda', '44 - La Guajira',
       '54 - Norte de Santander',
       '88 - Archipiélago de San Andrés, Providencia y Santa Catalina',
       '81 - Arauca', '91 - Amazonas', '99 - Vichada', '95 - Guaviare',
       '97 - Vaupés']

df[df.Departamento.isin(departamentos_colombia)]

,Fecha,Concepto_Desc,Capacidad,Departamento,Municipio,COVID_confirmados,COVID_sospechosos,No_COVID,Total_camas,Camas_disponibles,PCD,Total_casos_COVID,Porcentaje_camas_disponibles,Ocupación_total_COVID,Nombre_departamento,Nombre_municipio,PrefijoDpto,PrefijoMpio,FID,ID_CARGA
0,2020-07-01 12:00:00,CAMAS,Adultos,08 - Atlántico,08137 - Campo De La Cruz,0.0,0.0,0.0,7.0,7.0,1.000000,0.0,1.0000,0.0,Atlántico,Campo De La Cruz,el departamento de,", municipio de",1,NaN
1,2020-07-01 12:00:00,CAMAS,Adultos,08 - Atlántico,08296 - Galapa,0.0,0.0,0.0,10.0,10.0,1.000000,0.0,1.0000,0.0,Atlántico,Galapa,el departamento de,", municipio de",2,NaN
2,2020-07-01 12:00:00,CAMAS,Adultos,08 - Atlántico,08372 - Juan De Acosta,0.0,0.0,0.0,6.0,6.0,1.000000,0.0,1.0000,0.0,Atlántico,Juan De Acosta,el departamento de,", municipio de",3,NaN
3,2020-07-01 12:00:00,CAMAS,Adultos,08 - Atlántico,08421 - Luruaco,0.0,0.0,4.0,11.0,7.0,0.636364,0.0,0.6364,0.0,Atlántico,Luruaco,el departamento de,", municipio de",4,NaN
4,2020-07-01 12:00:00,CAMAS,Adultos,08 - Atlántico,08433 - Malambo,1.0,1.0,0.0,47.0,45.0,0.957447,2.0,0.9574,2.0,Atlántico,Malambo,el departamento de,", municipio de",5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438854,2023-11-26 12:00:00,CAMAS,Adultos,73 - Tolima,73319 - Guamo,0.0,0.0,0.0,10.0,10.0,1.000000,0.0,1.0000,0.0,Tolima,Guamo,el departamento de,", municipio de",1491233,NaN
1438855,2023-11-26 12:00:00,CAMAS,Adultos,73 - Tolima,73347 - Herveo,0.0,0.0,0.0,4.0,4.0,1.000000,0.0,1.0000,0.0,Tolima,Herveo,el departamento de,", municipio de",1491234,NaN
1438856,2023-11-26 12:00:00,CAMAS,Adultos,73 - Tolima,73349 - Honda,0.0,0.0,27.0,34.0,7.0,0.205882,0.0,0.2059,0.0,Tolima,Honda,el departamento de,", municipio de",1491235,NaN
1438857,2023-11-26 12:00:00,CAMAS,Adultos,73 - Tolima,73352 - Icononzo,0.0,0.0,0.0,4.0,4.0,1.000000,0.0,1.0000,0.0,Tolima,Icononzo,el departamento de,", municipio de",1491236,NaN
